In this notebook, we will see a very simple demonstration of using the MODFLOW-6 API capability for, you guessed, the Freyberg model...

First import as usual...

In [ ]:
import os
import shutil
import platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
import modflowapi


We can use pre-compiled mf6 binaries

In [ ]:
if "linux" in platform.platform().lower():
    lib_path = os.path.join("bin","linux","libmf6.so")
    exe_path = os.path.join("bin","linux","mf6")
    lib_name = os.path.split(lib_path)[-1]
    exe_name = "./"+os.path.split(exe_path)[-1]
elif "darwin" in platform.platform().lower() or "macos" in platform.platform().lower() :
    lib_path = os.path.join("bin","mac","libmf6.so")
    exe_path = os.path.join("bin","mac","mf6")
    lib_name = os.path.split(lib_path)[-1]
    exe_name = "./"+os.path.split(exe_path)[-1]
else:
    lib_path = os.path.join("bin","win","libmf6.dll")
    exe_path = os.path.join("bin","win","mf6.exe")  
    lib_name = os.path.split(lib_path)[-1]
    exe_name = os.path.split(exe_path)[-1]



Copy the original model files to a working dir and get the binaries into that dir

In [ ]:
org_dir = "monthly_model_files_org"
work_dir = "temp"
if os.path.exists(work_dir):
    shutil.rmtree(work_dir)
shutil.copytree(org_dir,work_dir)
shutil.copy2(lib_path,os.path.join(work_dir,os.path.split(lib_path)[-1]))
shutil.copy2(exe_path,os.path.join(work_dir,os.path.split(exe_path)[-1]))


Now just run standard mf6

In [ ]:
c_d = os.getcwd()
os.chdir(work_dir)
os.system(exe_name)
os.chdir(c_d)

Now lets see how the API is used - first just replicate the standard MF6 solution

In [ ]:
api_dir1 = "api1"
if os.path.exists(api_dir1):
    shutil.rmtree(api_dir1)
shutil.copytree(org_dir,api_dir1)
shutil.copy2(lib_path,os.path.join(api_dir1,os.path.split(lib_path)[-1]))
shutil.copy2(exe_path,os.path.join(api_dir1,os.path.split(exe_path)[-1]))

Create a `ModflowApi` instance for our model and initialize it

In [ ]:
gwf = modflowapi.ModflowApi(os.path.join(api_dir1, lib_name), working_directory=api_dir1)
gwf.initialize()

Now march thru the stress periods and solve as usual...

In [ ]:
# get current sim time
ctime = gwf.get_current_time()
# get ending sim time
etime = gwf.get_end_time()
# max number of iterations
max_iter = gwf.get_value(gwf.get_var_address("MXITER", "SLN_1"))
# let's do it!
while ctime < etime:
    # the length of this sim time
    dt = gwf.get_time_step()
    # prep the current time step
    gwf.prepare_time_step(dt)
    kiter = 0
    # prep to solve
    gwf.prepare_solve(1)
    # the current one-based stress period number
    stress_period = gwf.get_value(gwf.get_var_address("KPER", "TDIS"))[0]
    time_step = gwf.get_value(gwf.get_var_address("KSTP", "TDIS"))[0]
    # solve until converged
    while kiter < max_iter:
        if gwf.solve(1):
            print("flow stress period,time step {0},{1} converged with {2} iters".\
                  format(stress_period, time_step, kiter))
            break

        kiter += 1
    try:
        gwf.finalize_solve(1)
    except:
        pass

    gwf.finalize_time_step()
    # update current sim time
    ctime = gwf.get_current_time()
gwf.finalize()

Now let's do something more exciting.  We can use the API interface to build in operational rules for the pumping wells.  If the sfr downstream flow for the terminal reach is too low, we need to pump less, if the flow is high, we can pump more.

In [ ]:
api_dir2 = "api2"
if os.path.exists(api_dir2):
    shutil.rmtree(api_dir2)
shutil.copytree(org_dir,api_dir2)
shutil.copy2(lib_path,os.path.join(api_dir2,os.path.split(lib_path)[-1]))
shutil.copy2(exe_path,os.path.join(api_dir2,os.path.split(exe_path)[-1]))

In [ ]:
gwf = modflowapi.ModflowApi(os.path.join(api_dir2, lib_name), working_directory=api_dir2)
gwf.initialize()

Now we use march thru stess periods as before, but within the outer iteration loop, we will attempt some "management" to link the groundwater extraction rates to surface-water flow.  If the surface-water flow at the terminal downstream reach is less than 2,500, reduce the pumping rates.  Otherwise, if the surface-water flow is greater than 3,000, lets pump more!

In [ ]:
# get current sim time
ctime = gwf.get_current_time()
# get ending sim time
etime = gwf.get_end_time()
# max number of iterations
max_iter = gwf.get_value(gwf.get_var_address("MXITER", "SLN_1"))
# let's do it!
while ctime < etime:
    # the length of this sim time
    dt = gwf.get_time_step()
    # prep the current time step
    gwf.prepare_time_step(dt)
    kiter = 0
    # prep to solve
    gwf.prepare_solve(1)
    # the current one-based stress period number
    stress_period = gwf.get_value(gwf.get_var_address("KPER", "TDIS"))[0]
    time_step = gwf.get_value(gwf.get_var_address("KSTP", "TDIS"))[0]
    # solve until converged
    while kiter < max_iter:
        
        # get the sfr dsflow rate for the last reach
        addr = ["DSFLOW", "FREYBERG6", "SFR_1"]
        wbaddr = gwf.get_var_address(*addr)
        sfr_dsflow = gwf.get_value(wbaddr)
        
        #get the wel extraction rates
        "FREYBERG6/WEL-1                   BOUND"
        addr = ["BOUND", "FREYBERG6", "WEl-1"]
        wbaddr = gwf.get_var_address(*addr)
        wel_bound = gwf.get_value_ptr(wbaddr)
        org = np.sum(wel_bound)
        
        # if the flow is too low, turn down the wells
        fac = 1
        if sfr_dsflow[-1] < 2500:
            fac = sfr_dsflow[-1]/2500
            wel_bound[:] *= fac
        
        #otherewise if there is extra flow, turn up the wells
        elif sfr_dsflow[-1] > 3000:
            fac = sfr_dsflow[-1]/3000
            wel_bound[:] *= fac
        
        # if we have solved at least once and the solution is converged
        if gwf.solve(1) and kiter > 1:
            # get the sfr dsflow rate for the last reach
            addr = ["DSFLOW", "FREYBERG6", "SFR_1"]
            wbaddr = gwf.get_var_address(*addr)
            sfr_dsflow = gwf.get_value(wbaddr)
            print("flow stress period,time step {0},{1} converged with {2} iters, final sfr:{3:5.2f}".\
                  format(stress_period,time_step, kiter,sfr_dsflow[-1]))
            break
        
        
        print("  ---  iter:{0}, sfr:{1:5.2f} org wel:{2:5.2f} new wel:{3:5.2f} fac:{4:2.1f}" .\
              format(kiter,sfr_dsflow[-1],org,np.sum(wel_bound),fac))
        kiter += 1
    try:
        gwf.finalize_solve(1)
    except:
        pass
    
    
    gwf.finalize_time_step()
    # update current sim time
    ctime = gwf.get_current_time()
gwf.finalize()

Woah - that drastically increased the number of iterations - we are working the solver here with some strong nonlinearity!

In [ ]:
lst_org = flopy.utils.Mf6ListBudget(os.path.join(work_dir,"freyberg6.lst"))
lst_api = flopy.utils.Mf6ListBudget(os.path.join(api_dir2,"freyberg6.lst"))

In [ ]:
oinc,ocum = lst_org.get_dataframes(diff=True)
ainc,acum = lst_api.get_dataframes(diff=True)
ainc

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(10,10))
axes[0].plot(ainc.index,ainc.wel,color="r",label="api")
axes[0].plot(oinc.index,oinc.wel,color="c",label="org")
axes[1].plot(ainc.index,ainc.sfr,color="r",label="api")
axes[1].plot(oinc.index,oinc.sfr,color="c",label="org")
axes[0].legend()
axes[0].set_title("well flux")
axes[1].legend()
axes[1].set_title("sw-gw flux")


We can see the effect of our "management" strategy:  substantially more gw is extracted during the wet season while less groundwater is extracted during the dry season compared to the original simulation (that used constant extraction).  And we see that this contributes to less groundwater contribution to surface water during the wet season.